In [1]:

import sys
import argparse
# Add the parent directory to sys.path
sys.path.append('/storage/homefs/no21h426/DL-downscaling/')
import os
import yaml
import math
import warnings
import numpy as np
import xarray as xr
import dask
from time import time

# Import torch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torch.utils.data import Dataset, DataLoader, sampler, TensorDataset
from torch.utils.data import sampler
import torch.utils.checkpoint as checkpoint
import torch.nn.functional as F

import torchvision.datasets as dset
import torchvision.transforms as T

from einops import rearrange
from einops.layers.torch import Rearrange

# Utils
from utils.data_loader import *
from utils.utils_plot import *
from utils.utils_loss import *
from utils.helpers import *
from utils.Datagenerators import *
from src.constants import *
# Try dask.distributed and see if the performance improves...
from dask.distributed import Client
c = Client(n_workers=os.cpu_count()-2, threads_per_worker=1)

warnings.filterwarnings("ignore", category=RuntimeWarning, message="divide by zero encountered in divide")

/storage/homefs/no21h426/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print("Cuda Avaliable :", torch.cuda.is_available())

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

Cuda Avaliable : True
cuda


In [3]:
target = load_target_data(DATE_START, DATE_END, PATH_MCH)
x_axis = target.TabsD.x
y_axis = target.TabsD.y
    

input_data = load_input_data(DATE_START, DATE_END, PATH_DEM, INPUT_VARIABLES, INPUT_PATHS, 
                                 LEVELS, RESOL_LOW, x_axis, y_axis)



if DO_CROP:
    input_data = input_data.sel(x=slice(min(CROP_X), max(CROP_X)), y=slice(max(CROP_Y), min(CROP_Y)))
    target = target.sel(x=slice(min(CROP_X), max(CROP_X)), y=slice(max(CROP_Y), min(CROP_Y)))


# Split the data (small data for testing purposes)
x_train = input_data.sel(time=slice('1999', '2011')) 
x_valid = input_data.sel(time=slice('2012', '2015')) 
x_test = input_data.sel(time=slice('2016', '2021'))

y_train = target.sel(time=slice('1999', '2011'))
y_valid = target.sel(time=slice('2012', '2015'))
y_test = target.sel(time=slice('2006', '2011'))

# Select the variables to use as input and output
input_vars = {'topo' : None, 'tp': None, 't': LEVELS}
output_vars = ['RhiresD', 'TabsD'] #['RhiresD', 'TabsD', 'TmaxD', 'TminD']

training_set = DataGenerator(x_train, y_train, input_vars, output_vars)
loader_train = torch.utils.data.DataLoader(training_set, batch_size=32)

# Validation
valid_set = DataGenerator(x_valid, y_valid, input_vars, output_vars, shuffle=False, mean=training_set.mean, std=training_set.std)
loader_val = torch.utils.data.DataLoader(valid_set, batch_size=32)

# Test
test_set = DataGenerator(x_test, y_test, input_vars, output_vars, shuffle=False, mean=training_set.mean, std=training_set.std)
loader_test = torch.utils.data.DataLoader(test_set, batch_size=32)

torch.cuda.empty_cache()

Target data loaded from pickle.
Input data loaded from pickle.


/storage/homefs/no21h426/.local/lib/python3.10/site-packages/distributed/client.py:3109: UserWarning: Sending large graph of size 115.97 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
/storage/homefs/no21h426/.local/lib/python3.10/site-packages/distributed/client.py:3109: UserWarning: Sending large graph of size 115.97 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


Loading data into RAM


/storage/homefs/no21h426/.local/lib/python3.10/site-packages/distributed/client.py:3109: UserWarning: Sending large graph of size 115.97 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


Loading data into RAM


/storage/homefs/no21h426/.local/lib/python3.10/site-packages/distributed/client.py:3109: UserWarning: Sending large graph of size 35.69 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


Loading data into RAM


/storage/homefs/no21h426/.local/lib/python3.10/site-packages/distributed/client.py:3109: UserWarning: Sending large graph of size 53.54 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
